In [1]:
using BenchmarkTools
using Base.Test

In [11]:
module nowrap

import Base: getindex

type Statistics
    word::String
end

abstract Feature
size(::Feature) = ()
getindex(f::Feature, ::CartesianIndex{0}) = f

const FEATURES = Feature[]

function allfeatures()
    return FEATURES
end

macro addfeatures(T, expr)
    quote
        allfeatures(::Type{$T}) = $(esc(expr))
        append!(FEATURES, allfeatures($T))
    end
end

immutable ContainsLetter <: Feature
    letter::Char
end
@addfeatures ContainsLetter [ContainsLetter(l) for l in 'a':'z']
satisfies(f::ContainsLetter, stat) = f.letter in stat.word
# (f::ContainsLetter)(stat) = f.letter in stat.word

immutable LetterAtIndex <: Feature
    letter::Char
    index::Int
end
@addfeatures LetterAtIndex [LetterAtIndex(l, j) for l in 'a':'z' for j in 1:26]
satisfies(f::LetterAtIndex, stat) = length(stat.word) >= f.index && stat.word[f.index] == f.letter
# (f::LetterAtIndex)(stat) = length(stat.word) >= f.index && stat.word[f.index] == f.letter

end

nowrap

In [12]:
stat = nowrap.Statistics("hello")
fs = nowrap.allfeatures()
@benchmark [nowrap.satisfies(f, $stat) for f in $fs]

BenchmarkTools.Trial: 
  memory estimate:  848.00 bytes
  allocs estimate:  3
  --------------
  minimum time:     34.332 μs (0.00% GC)
  median time:      37.552 μs (0.00% GC)
  mean time:        42.055 μs (0.00% GC)
  maximum time:     231.012 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [13]:
f = fs[1]
@benchmark nowrap.satisfies($f, $stat)

BenchmarkTools.Trial: 
  memory estimate:  0.00 bytes
  allocs estimate:  0
  --------------
  minimum time:     21.617 ns (0.00% GC)
  median time:      23.715 ns (0.00% GC)
  mean time:        29.782 ns (0.00% GC)
  maximum time:     181.680 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     997
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [14]:
@benchmark [f($stat) for f in $fs]

BenchmarkTools.Trial: 
  memory estimate:  864.00 bytes
  allocs estimate:  4
  --------------
  minimum time:     25.161 μs (0.00% GC)
  median time:      25.818 μs (0.00% GC)
  mean time:        27.955 μs (0.00% GC)
  maximum time:     140.912 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [15]:
@benchmark $f($stat)

BenchmarkTools.Trial: 
  memory estimate:  0.00 bytes
  allocs estimate:  0
  --------------
  minimum time:     21.570 ns (0.00% GC)
  median time:      23.747 ns (0.00% GC)
  mean time:        24.509 ns (0.00% GC)
  maximum time:     123.153 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     996
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [16]:
[f(stat) for f in fs]

702-element Array{Bool,1}:
 false
 false
 false
 false
  true
 false
 false
  true
 false
 false
 false
  true
 false
     ⋮
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false

In [4]:
f = fs[1]

nowrap.ContainsLetter('a')

In [6]:
@code_warntype nowrap.satisfies.(fs, stat)

Variables:
  #self#::Base.Broadcast.#broadcast
  f::nowrap.#satisfies
  As::Tuple{Array{nowrap.Feature,1},nowrap.Statistics}

Body:
  begin 
      return $(Expr(:invoke, LambdaInfo for broadcast_t(::Function, ::Type{Any}, ::Array{nowrap.Feature,1}, ::Vararg{Any,N}), :(Base.Broadcast.broadcast_t), :(f), :($(QuoteNode(Any))), :((Core.getfield)(As,1)::Array{nowrap.Feature,1}), :((Core.getfield)(As,2)::nowrap.Statistics)))
  end::ANY
